# Magic Squares
---
_**Source**: Slides "Constraint programming 1 - Modelling with OR-Tools"_  
_**Author**: Adrian Kauz_

>A magic square is an arrangement of distinct integers in a square grid, such that the values in each row, in each column and in the two main diagonals all add up to the same number. If $n$ denotes a number of cells, the value 1 to $n$ are to be distributed.

![MagicSquares](images/MagicSquares.png)

>Write an OR-TOOLS model for _Magic Squares_ of arbitary size.

## Imports

In [1]:
from ortools.constraint_solver import pywrapcp
import itertools
import numpy as np

## Helper functions

In [2]:
def printSolution(currentSolution):
    print("Possible solution:")
    
    for row in range(len(currentSolution)):
        currentRow = ""
        for column in range(len(currentSolution[row])):
            currentRow += " {:02d}".format(currentSolution[row][column].Value())
            
        print(currentRow)
        
    print("")

## Solution

In [3]:
# Size of the Magic Square
boardSize = 4

solver = pywrapcp.Solver("MagicSquares")

# Calculate magic number
magicNumber = int(abs((pow(boardSize, 3) + boardSize) / 2))
print("Magic number: {}\n".format(magicNumber))

# n x n Matrix of Decision Variables
board = [[solver.IntVar(1, boardSize * boardSize) for _j in range(boardSize)] for _i in range(boardSize)]


# --------------------------------------------------------------------------------
# Set constraints
# --------------------------------------------------------------------------------
# All squares different
solver.Add(solver.AllDifferent(list(np.concatenate(board))))

# Each row/column should equal the magic number
for i in range(boardSize):
    solver.Add(solver.Sum([board[i][j] for j in range(boardSize)]) == magicNumber) # Rows
    solver.Add(solver.Sum([board[j][i] for j in range(boardSize)]) == magicNumber) # Columns

# Same for the diagonal part
solver.Add(solver.Sum([board[j][j] for j in range(boardSize)]) == magicNumber)                 # Top/Left to Bottom/Right
solver.Add(solver.Sum([board[j][boardSize - 1 - j] for j in range(boardSize)]) == magicNumber) # Top/Right to Bottom/Left


# --------------------------------------------------------------------------------
# Now solve it!
# --------------------------------------------------------------------------------
db = solver.Phase(list(np.concatenate(board)), # Sudoku board as array
                solver.INT_VAR_SIMPLE,         # Variable selection policy for search
                solver.INT_VALUE_SIMPLE)       # Value selection policy for search

solver.NewSearch(db)

if solver.NextSolution():
    printSolution(board)
    
solver.EndSearch()

#Display the number of solutions found:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime: {}ms".format(solver.WallTime()))
print("Failures: {}".format(solver.Failures()))
print("Branches: {}".format(solver.Branches()))

Magic number: 34

Possible solution:
 01 02 15 16
 12 14 03 05
 13 07 10 04
 08 11 06 09

Solutions: 1
Runtime: 2ms
Failures: 7
Branches: 18
